In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import pandas as pd
import re

import requests
import requests.auth

import praw

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

praw.__version__

'7.7.0'

See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 
 needs a .env file per dot-env-template
 

In [2]:
csv_validate_re = re.compile(r'''
    \s*                # Any whitespace.
    (                  # Start capturing here.
      [^,"']+?         # Either a series of non-comma non-quote characters.
      |                # OR
      "(?:             # A double-quote followed by a string of characters...
          [^"\\]|\\.   # That are either non-quotes or escaped...
       )*              # ...repeated any number of times.
      "                # Followed by a closing double-quote.
      |                # OR
      '(?:[^'\\]|\\.)*'# Same as above, for single quotes.
    )                  # Done capturing.
    \s*                # Allow arbitrary space before the comma.
    (?:,|$)            # Followed by a comma or the end of a string.
    ''', re.VERBOSE)


In [3]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

# a prompt to apply to replies on the thread
# prefix = """Define an example CSV file output as follows: 
# "artist","song_title"
# "The Beatles","Yesterday"
# "Eagles","Hotel California"

# Extract all song titles and artists from the following input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:
# """


system_prompt="You will act as a research assistant finding all the artists and track titles in a document."
assistant_prompt="""Define a comma-separated values output as follows: 
artist,track
"The Beatles","Yesterday"
"Eagles","Hotel California" """
user_prefix="""You will extract all artists and tracks from the following input, and return a list of records containing each artist and track extracted from the input in a comma-separated values format. The header row should contain `"artist", "track"`. The fields in each record should be enclosed in double-quotes. The input is:"""
# an output file to accumulate all the responses
savefile = 'bronze.txt'


## Get all comments from a reddit posting

In [4]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [5]:
# print(datetime.now())
# r = getPraw()
# res = getAll(r, submission)
# print(datetime.now())

# print("retrieved ", len(res), 'comments')

In [6]:
# # we have a list of comment objects
# # filter comments with at least some karma
# res3 = [r for r in res if r.score >= minkarma]
# print('filtered to ', len(res3), 'comments')
# res3[0].body, res3[0].score

In [7]:
# save so we can reload it later without downloading

# with open('reddit.pkl', 'wb') as f:
#     pickle.dump(res3, f)
    
with open('reddit.pkl', 'rb') as f:
    res3 = pickle.load(f)
    

## Extract artists and song titles using OpenAI

In [8]:
# check lengths of posts
shorties = []
big_ones = []
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
        shorties.append(i)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        big_ones.append(i)
        

474 4162
1539 W


In [9]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)

105.34075546719681

In [10]:
print (res3[big_ones[0]].body[:500])

Saturn by Sleeping at Last:
https://www.youtube.com/watch?v=dzNvk80XY9s

The version they did with Tim Fain is even more beautiful: 
https://www.youtube.com/watch?v=0nRpeAiur9Q

I'm not good at choosing one thing from a list of favorites as the best, so I've got about 30+ answers that are really a 30+ -way tie, and the one that I would consider as "prettiest" at any given moment is heavily influenced by my current mood. So, it could be any one of these from my "Heart Wrenchingly Beautiful" playl


In [12]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

openai.api_key = os.getenv('OPENAI_API_KEY')

slist = res3.copy()

# to speed things we'll cumulate posts til we get to 100 posts or 5000 chars, whichever comes first
maxchars = 5000  # max tokens is 4096 but we'll limit each prompt to 5000 chars
nposts = 100

# make sure no single post > maxchars + prefix which breaks the logic below
for i in range(len(slist)):
    if len(slist[i].body) > maxchars + len(user_prefix):
        print ("truncated ", i)
        slist[i].body = slist[i].body[:maxchars + len(user_prefix)]
        
outdir = 'out'
logdir = 'logs'
# make sure out and logs are empty
for f in glob.glob('%s/*' % outdir):
    os.remove(f)
for f in glob.glob('%s/*' % logdir):
    os.remove(f)
count = 0
c = 0


while(slist):  # still comments to process
    prompt = ""
    reply_ids = []

    for _ in range(nposts):  # add up to 100 posts to the prompt
        if slist:
            if len(prompt) + len(slist[0].body) < maxchars:
                reply = slist.pop(0)
                reply_ids.append(reply.id)
                body = reply.body # in order, for better context 
#                 if len(body) <3:
#                     print (c, body)
                prompt += body
                prompt += " \n \n"
#                 c += 1            
            
    # retry loop, have received untrapped 502 error
    for _ in range(3):
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-0301',
                messages=[{"role":"system", "content": system_prompt},
                          {"role":"assistant", "content": assistant_prompt},
                          {"role":"user", "content": user_prefix + prompt}
                         ],
                temperature=0,
            )
        except Exception as error:
            print("An exception occurred:", error)
            print("looping...")
            time.sleep(5)
            continue  # try again
        break   # exception not triggered

    # do basic validation and cleanup
    csv_output = response['choices'][0]['message']['content']
    csv_valid, csv_err = [], []
    for line in csv_output.split("\n"):
        try:
            csv_values = csv_validate_re.findall(line)
            if len(csv_values) == 2:
                csv_valid.append(line)
            else:
                csv_err.append(line)
        except:
            csv_err.append(line)
    csv_output = "\n".join(csv_valid)
        
    with open("%s/%04d.csv" % (outdir, count), 'w') as outfile:
        outfile.write(csv_output)
    
    if csv_err:
        with open("%s/%04d.err" % (outdir, count), 'w') as outfile:
            outfile.write("\n".join(csv_err))
        
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(str(reply_ids))
        logfile.write('\n\n===== raw prompt =====\n\n')        
        logfile.write(prompt)
        logfile.write('\n\n===== raw response =====\n\n')
        logfile.write(response['choices'][0]['message']['content'])
        if csv_err:
            logfile.write('\n\n===== failed validation =====\n\n')
            logfile.write("\n".join(csv_err))
 
    count += 1
#     print(c)
    print(count, end=' ')
    
print()
print(datetime.now())



2023-05-03 13:10:29.347952
..........................................................
2023-05-03 13:30:38.576320


In [29]:
# may still have to tweak the files to get them to load
# should inspect .err files and clean up if possible

filelist = glob.glob('%s/*.csv' % outdir)

output_df = None

for f in sorted(filelist):
    print(f)
    try:
        tempdf = pd.read_csv("%s" % (f), header=None)
    except Exception as exc:
        print(str(exc))
        continue
    colcount = len(tempdf.columns)
    if len(tempdf.columns) != 2:
        print('%s has %d columns, skipped' % (f, colcount))
        continue
    # ok
    # truncate header row
    if tempdf.iloc[0][0]=='artist' and tempdf.iloc[0][1]=='track':
        tempdf = tempdf[1:]
    if output_df is not None:        
        output_df = pd.concat([output_df, tempdf], axis=0)
    else:
        output_df = tempdf
output_df.columns=['artist', 'track']

out/0000.csv
out/0001.csv
out/0002.csv
out/0003.csv
out/0004.csv
out/0005.csv
out/0006.csv
out/0007.csv
out/0008.csv
out/0009.csv
out/0010.csv
out/0011.csv
out/0012.csv
out/0013.csv
out/0014.csv
out/0015.csv
out/0016.csv
out/0017.csv
out/0018.csv
out/0019.csv
out/0020.csv
out/0021.csv
out/0022.csv
out/0023.csv
out/0024.csv
out/0025.csv
out/0026.csv
out/0027.csv
out/0028.csv
out/0029.csv
out/0030.csv
out/0031.csv
out/0032.csv
out/0033.csv
out/0034.csv
out/0035.csv
out/0036.csv
out/0037.csv
out/0038.csv
out/0039.csv
out/0040.csv
out/0041.csv
out/0042.csv
out/0043.csv
out/0044.csv
out/0045.csv
out/0046.csv
out/0047.csv
out/0048.csv
out/0049.csv
out/0050.csv
out/0051.csv
out/0052.csv
out/0053.csv
out/0054.csv
out/0055.csv
out/0056.csv
out/0057.csv


In [31]:
output_df


,artist,track
1,Erik Satie,Gymnopédies
2,Don McLean,"Vincent (Starry, Starry Night)"
3,Lord Huron,The night we met
4,Neil Young,Harvest Moon
5,Simon & Garfunkel,Scarborough Fair
...,...,...
6,Apocalyptica,NaN
7,Nightwish,NaN
1,Felix Mendelssohn,Songs Without Words
2,Benny Goodman,"Sing, Sing, Sing"


In [32]:
# save bronze
output_df.to_csv(savefile, index=False)
len(output_df)

1547

In [34]:
df = pd.read_csv(savefile) \
    .drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"]) \
    .reset_index(drop=True)

df.to_csv('silver.csv', index=False)

print(len(df))
# tweak further to get to gold.csv

df


1249


,artist,track
0,311,Amber
1,A Perfect Circle,Orestes
2,A Perfect Circle,Three Libras
3,A-Ha,Take on Me
4,ABBA,Fernando
...,...,...
1244,unknown,Pachelbel rant
1245,unknown,"Stop this heartbreak overload, I ain't missing..."
1246,unknown,Tin Man
1247,unknown,unknown


In [35]:
df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('track', ascending=False) \
    .head(10)



,artist,track
547,Unknown,66
74,Bob Dylan,36
410,Radiohead,26
482,The Beatles,26
200,George Harrison,24
312,Leonard Cohen,16
493,The Cure,16
161,Enya,16
445,Simon & Garfunkel,16
446,Simon and Garfunkel,15


In [36]:
df = df.drop(df.loc[df['artist']=='Unknown'].index)
df = df.drop(df.loc[df['artist']=='unknown'].index)

In [38]:
df.groupby('track') \
    .count() \
    .reset_index() \
    .sort_values('artist', ascending=False) \
    .head(10)


,track,artist
1051,unknown,12
331,Hallelujah,6
842,The Boxer,5
477,La Vie En Rose,5
1052,version,4
487,Landslide,3
936,Unknown,3
263,Falling Slowly,3
728,Scarborough Fair,3
909,Time,3


In [39]:
df = df.drop(df.loc[df['track']=='Unknown'].index)
df = df.drop(df.loc[df['track']=='unknown'].index)

In [40]:
len(df)

1162

In [41]:
df.to_csv('silver.csv', index=False)


## Load into a Spotify playlist


In [42]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [44]:
# check artists
df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe = {}
fail_list = []

for index, artist, title in df.itertuples():
    if artist in dedupe:
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(artist, artist_names)
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

# then clean up manually as appropriate

A-Ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
Adeem ['Adeem the Artist', 'Adeem', 'Adeema']
Al Stewart ['Alexander Stewart', 'Al Stewart', 'Alec Lee-Stewart']
Alan Parsons Project ['The Alan Parsons Project', 'The Alan Parsons Symphonic Project', 'The Alan Parsons Tribute Project']
Allegri ['Gregorio Allegri', 'Ensemble Allegria', 'Piero Allegrini']
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Krauss - Whiskey Lullaby
Allman brothers ['Allman Brothers Band', 'The Allman Brothers Band', 'The Allman Brothers Band With Sheryl Crow']
America ['The All-American Rejects', 'America', 'American Authors']
Aretha ['Aretha Franklin', 'Aretha Grant', 'Aretha Scruggs']
Babyface ['Babyface Ray', 'Babyface', 'BabyFaceWood']
Bach ['Johann Sebastian Bach', 'Bachman-Turner Overdrive', 'Tal Bachman']
Barbara ['Ana Bárbara', 'Barbara Mandrell', 'Barbara Mason']
Barber ['Sam Barber', 'Jill Barber', 'Samuel Barber']
not found: Barry De Vorzon and Perry Botkin Jr. - Nadi

Sundays ['The Sundays', 'Sarah and the Sundays', 'Sundays By The Ocean']
Tchaikovsky ['Pyotr Ilyich Tchaikovsky', 'Tchaikovsky Chamber Orchestra', 'The Tchaikovsky Symphony Orchestra']
not found: Terrence Jay - One blood
The Band ['The Band CAMINO', 'The Band', 'The Charlie Daniels Band']
The Boxer ['The Boxer Rebellion', 'The Boxer', 'Gladys and the Boxers']
not found: The Cinematic Orchestra, Patrick Watson - To Build A Home
not found: The Cocteau Twins - Anything that Liz Fraser sings
The Fugees ['The Re-Fugees']
The Grateful Dead ['Grateful For The Dead']
The King’s Singers ["The King's Singers", 'The Kingdom Singers', 'The Bob King Singers']
The La’s ['The Kid LAROI', 'The Lacs', 'The Laurie Berkner Band']
The Mamas and the Papas ['The Mamas & The Papas', 'The Mamas & The Papas Experience', 'Karaoke - The Mamas & The Papas']
The OutKast ['The Outkasts', 'The Crazy Outkasts', 'KHALIDD THE OUTKAST']
The Sunday's ['The Sundays', 'Sarah and the Sundays', 'The Sunday Shakes']
Trent Rez

In [45]:
# check tracks

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

A Perfect Circle|Three Libras : A Perfect Circle|3 Libras
not found: Across the Universe - Here Comes the Sun
not found: Adeem - White Trash Revelry
not found: Adeem the Artist - The Great Unknown
not found: Allison Kraus - When You Say Nothing at All
not found: Allison Krauss - Whiskey Lullaby
not found: Alter Bridge - Dreams of a Hero
not found: Alter Bridge - Why not both?
Alter Bridge|version : Alter Bridge|Rise Today - Single Version
Andy Williams|Moon River : Andy Williams|Moon River (From "Breakfast at Tiffany's")
not found: Aphex Twin - Avril 14th/QKThr
not found: Aphex Twin - Avrl 14
Aphex Twin|Come to Daddy : Aphex Twin|Come To Daddy - Pappy Mix
Aphex Twin|IZ Us : Aphex Twin|IZ-US
not found: Aphex Twin - Lichen
Aphex Twin|Rhubarb : Aphex Twin|Donkey Rhubarb
not found: Aphex Twin - Stone in Focus
not found: Aphex Twin - Untitled #17
not found: Aretha - version is beautiful if you haven’t heard it
Arvo Pärt|Spiegel im Spiegel : Arvo Pärt|Spiegel im Spiegel - Version for Violin 

Fleetwood Mac|The Dance : Fleetwood Mac|The Chain - Live at Warner Brothers Studios in Burbank, CA 5/23/97
not found: Flight Facilities - Claire de Lune
Foo Fighters|Everlong (Acoustic) : Foo Fighters|Everlong - Acoustic Version
not found: Frank Ocean - Rushes
Frank Sinatra|Fly Me to the Moon : Frank Sinatra|Fly Me To The Moon (In Other Words)
Franz Schubert|Ave Maria : Franz Schubert|Ave Maria, D. 839
not found: Franz Schubert - Prelude 12/21
Frederic Chopin|Nocturne No. 2 : Frédéric Chopin|Chopin: Nocturne No. 20 in C-Sharp Minor, Op. Posth.
Frente|Bizarre Love Triangle : Frente!|Bizarre Love Triangle - 2014 Remaster
Gabriel|In Your Eyes : Peter Gabriel|In Your Eyes - 2012 Remaster
George Harrison|All Things Must Pass : George Harrison|All Things Must Pass - 2014 Remaster
George Harrison|Apple Scruffs : George Harrison|Apple Scruffs - 2014 Remaster
George Harrison|Art of Dying : George Harrison|Art of Dying - 2014 Remaster
George Harrison|Awaiting on You All : George Harrison|Awaitin

Kate Bush|Wuthering Heights : Kate Bush|Wuthering Heights - 2018 Remaster
not found: Kozue Takada - Himitsu Kichi
Lauryn Hill|Ex factor : Ms. Lauryn Hill|Ex-Factor
Led Zeppelin|All My Love : Led Zeppelin|All My Love - Remaster
Led Zeppelin|Going to California : Led Zeppelin|Going to California - Remaster
Led Zeppelin|Rain Song : Led Zeppelin|The Rain Song - Remaster
Led Zeppelin|Thank you : Led Zeppelin|Thank You - 1990 Remaster
Leo Delibes|Lakme : Léo Delibes|Delibes: Lakmé, Act 1: "Dôme épais, le jasmin" (Lakmé, Mallika)
Leonard Cohen|Chelsea Hotel : Leonard Cohen|Chelsea Hotel #2
not found: Leonard Cohen - I'm Your Man
not found: Leonard Cohen - New Skin for the Old Ceremony
not found: Leonard Cohen - Old Ideas
not found: Leonard Cohen - Popular Problems
not found: Leonard Cohen - Recent Songs
not found: Leonard Cohen - Songs from a Room
not found: Leonard Cohen - Songs of Leonard Cohen
not found: Leonard Cohen - Songs of Love and Hate
not found: Leonard Cohen - Ten New Songs
not fo

not found: Simon and Garfunkel - For Emily Wherever I May Find Her
not found: Simon and Garfunkel - She Was My True Love
Simon and Garfunkel|Sounds of Silence : Simon & Garfunkel|The Sounds of Silence
Simon and Garfunkel|The 59th Street Bridge Song : Simon & Garfunkel|The 59th Street Bridge Song (Feelin' Groovy)
Sky Cries Mary|Sky Cries Mary : Sky Cries Mary|Don't Forget The Sky
Smashing Pumpkins|1979 : The Smashing Pumpkins|1979 - Remastered 2012
not found: South Park - Jacking it in San Diego
Steven Universe|Here comes a thought : Steven Universe|Here Comes a Thought (feat. Estelle & AJ Michalka)
Stevie Wonder|Isn't She Lovely : Stevie Wonder|Isn’t She Lovely
Sufjan Stevens|Mystery of Love : Sufjan Stevens|Mystery of Love (From the Original Motion Picture “Call Me by Your Name”)
not found: Sundays - You’re Not The Only One I Know
Taylor Swift|Fearless : Taylor Swift|Fearless (Taylor’s Version)
Taylor Swift|The Lakes : Taylor Swift|the lakes - bonus track
Tchaikovsky|Waltz of the Flow

In [46]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Orestes,Orestes,Mer De Noms,spotify:track:6YUKAH1icwkA3U7fVp3amo
2,A Perfect Circle,A Perfect Circle,Three Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
3,A-Ha,a-ha,Take on Me,Take on Me,Hunting High and Low,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
4,ABBA,ABBA,Fernando,Fernando,Arrival,spotify:track:4BM8yJ0PzBi2ZewpMTOxtx
...,...,...,...,...,...,...
857,beabadoobee,beabadoobee,glue,Glue Song,Glue Song,spotify:track:3iBgrkexCzVuPy4O9vx7Mf
858,beyonce,Beyoncé,Plastic off the sofa,PLASTIC OFF THE SOFA,RENAISSANCE,spotify:track:6ufcuVInt0ocHrUimDjGlb
859,lil Yachty,Lil Yachty,Drive me crazy,drive ME crazy!,Let’s Start Here.,spotify:track:6luBKkFUt5wTwz7hpLhp12
860,paper kites,The Paper Kites,Bloom,Bloom - Bonus Track,Woodland,spotify:track:1HMQmOWrkieKYWlFsjUP3D


In [47]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])

,input_artist,artist,input_track,track,album,uri
11,Alan Parsons Project,The Alan Parsons Project,Old and Wise,Old and Wise,Eye In The Sky (Expanded Edition),spotify:track:5Jt2AQv1c3RUF5ENtAYF1i
12,Alan Parsons Project,The Alan Parsons Project,Silence and I,Silence and I,Eye In The Sky (Expanded Edition),spotify:track:1MXd1awM0A7T7FxUPKAujf
13,Alan Parsons Project,The Alan Parsons Project,Time,Time,The Turn Of A Friendly Card (Expanded Edition),spotify:track:48yJZwYYDZX5GKFND7wDfC
16,Alison Krauss,Alison Krauss & Union Station,New Favorite,New Favorite,New Favorite,spotify:track:4iVeKORDsSUxkLzmme2bbL
18,Allegri,Gregorio Allegri,"Miserere Mei, Deus","Miserere mei, Deus",Allegri - Miserere,spotify:track:6es7DmrhnDoKj5rsFvh3XU
19,Allman brothers,Allman Brothers Band,Little Martha,Little Martha,Eat A Peach,spotify:track:2WPLFvAldG0GG6Ad3Xa0TO
25,America,American Folk Channel,The Boxer,The Boxer,American Country Folk for the Trucks 2,spotify:track:6jLXjtanB0Yp82aAV7VGsN
50,Barber,Samuel Barber,Adagio for Strings,Barber: Adagio for Strings,Samuel Barber - Adagio,spotify:track:1CSaCKPIp2yCIDL3t7Fyau
53,Beach Boys,The Beach Boys,Disney Girls,Disney Girls (1957) - Remastered 2009,Surf's Up,spotify:track:17qLiavc8woWHAPAGZD0Py
54,Beach Boys,The Beach Boys,In My Room,In My Room - Remastered,Surfer Girl (Remastered),spotify:track:62fX8EW16l8St2yL8rMer9


0           amber
1         orestes
2        3 libras
3      take on me
4        fernando
          ...    
834     glue song
835    plastic of
836    drive me c
837    bloom - bo
838    happiness 
Name: track, Length: 839, dtype: object

In [63]:
gold_df2=gold_df.copy().reset_index(drop=True)
gold_df2['input_track']=gold_df2['input_track'].str.lower()
gold_df2['input_track']=gold_df2['input_track'].apply(lambda s: s.strip()[:10])

gold_df2['track']=gold_df2['track'].str.lower()
gold_df2['track']=gold_df2['track'].apply(lambda s: s.strip()[:10])


with pd.option_context("display.max_rows", 999):
    display(gold_df2.loc[gold_df2['input_track'] != gold_df2['track']])

,input_artist,artist,input_track,track,album,uri
2,A Perfect Circle,A Perfect Circle,three libr,3 libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
21,Alter Bridge,Alter Bridge,version,rise today,Rise Today,spotify:track:7kPAtuZpG5zZGxd4avfYVB
40,Aphex Twin,Aphex Twin,iz us,iz-us,Come To Daddy,spotify:track:6INSqTqDoz6ndc0oKTi0Sq
41,Aphex Twin,Aphex Twin,rhubarb,donkey rhu,Donkey Rhubarb,spotify:track:3IfmatoAjPtQOqteemdUnp
49,Barber,Samuel Barber,adagio for,barber: ad,Samuel Barber - Adagio,spotify:track:1CSaCKPIp2yCIDL3t7Fyau
51,Basil Poledouris,Basil Poledouris,theology/c,conan the,Conan the Barbarian (Arr. P. Pelster),spotify:track:2K6NHYgiXgMsP27zjhxaaT
54,Beach Boys,The Beach Boys,surf’s up,surf's up,Surf's Up,spotify:track:5YniFjdw9nU8jCzvWlVVQC
61,Beethoven,Ludwig van Beethoven,9th sympho,a loud bit,Shine - Original Motion Picture Soundtrack,spotify:track:0AXEKshfGLZfJhpyV8KPbB
62,Beethoven,Ludwig van Beethoven,für elise,bagatelle,"Für Elise, Bagatelle No. 25 in A Minor, WoO 59",spotify:track:3zLTPuucd3e6TxZnu2dlVS
63,Beethoven,Ludwig van Beethoven,moonlight,sonata no.,"Beethoven: Piano Sonatas, Vol.3",spotify:track:3DNRdudZ2SstnDCVKFdXxG


In [66]:
# these are songs that look like covers or otherwise not the expected response from spotify search 
# (which is a bit wonky, doesn't like quotes and such)

bad_lookups = [
#    25,134,155,160,200,209,422,445,446,557,737,744,755,759,760,761,762,781,785,790,814,815,842
    21,51,61,63,83,145,212,317,322,439,449,575,759,784,
]

for i in bad_lookups:
    print(gold_df.iloc[i])
    
# add manually, plus 'not found'


input_artist                            Alter Bridge
artist                                  Alter Bridge
input_track                                  version
track                    Rise Today - Single Version
album                                     Rise Today
uri             spotify:track:7kPAtuZpG5zZGxd4avfYVB
Name: 21, dtype: object
input_artist                                     Basil Poledouris
artist                                           Basil Poledouris
input_track                                 Theology/Civilization
track           Conan the Barbarian (arr. P. Pelster for organ...
album                       Conan the Barbarian (Arr. P. Pelster)
uri                          spotify:track:2K6NHYgiXgMsP27zjhxaaT
Name: 51, dtype: object
input_artist                                            Beethoven
artist                                       Ludwig van Beethoven
input_track                                          9th symphony
track           A Loud Bit Of Ludwig's 9

In [67]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Orestes,Orestes,Mer De Noms,spotify:track:6YUKAH1icwkA3U7fVp3amo
2,A Perfect Circle,A Perfect Circle,Three Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
3,A-Ha,a-ha,Take on Me,Take on Me,Hunting High and Low,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
4,ABBA,ABBA,Fernando,Fernando,Arrival,spotify:track:4BM8yJ0PzBi2ZewpMTOxtx
5,ABBA,ABBA,One of us,One Of Us,The Essential Collection,spotify:track:6zgtBUEkAfilJ2YEOvNexR
6,ABBA,ABBA,The winner takes it all,The Winner Takes It All,Super Trouper,spotify:track:3oEkrIfXfSh9zGnE7eBzSV
7,Aaron Copland,Aaron Copland,Fanfare for the Common Man,Fanfare for the Common Man,Copland: Super Hits,spotify:track:0e6c45OSjdtl0Jew8dngDQ
8,Adrián Berenguer,Adrián Berenguer,Moonlight,Moonlight,The Journey,spotify:track:1vHan1eqIuiBG5jfliGO65
9,Aerosmith,Aerosmith,Dream On,Dream On,Aerosmith,spotify:track:1xsYj84j7hUDDnTTerGWlH


In [68]:
# get playlist id
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        playlist_id = playlist['id']
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

08YFkbtTV6GBfNtjJ4PHDu
   1 spotify:playlist:08YFkbtTV6GBfNtjJ4PHDu Reddit Prettiest Songs


In [69]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [ ]:
# addlist = gold_df['uri'].to_list()
# print (len(addlist))

# while(addlist):
#     sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
#                                 playlist_id=playlist_id, 
#                                 tracks=addlist[-100:])
#     addlist = addlist[:-100]
#     print("added items, remaining ", len(addlist))


In [ ]:
# manually add the ones that weren't found for some reason


In [70]:
# can run again and add any new tracks, either because OpenAI is a bit random, or new replies in thread
results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), playlist_id,
                                fields='tracks,next,name')
tracks = results['tracks']

playlist_dict_by_uri = {}
playlist_dict_by_str = {}

artist_list = []
track_list = []
uri_list = []
popularity_list = []
album_list=[]

while True:
    for track_item in tracks['items']:
        track_dict = track_item['track']
        track_str = track_dict['artists'][0]['name']  + ' | ' + track_dict['name'][:15]
        uri = track_dict['uri']
        if track_str in playlist_dict_by_str:
            print(track_str)
        playlist_dict_by_str[track_str] = uri
        playlist_dict_by_uri[uri] = track_str
        
        uri_list.append(uri)
        artist_list.append(track_dict['artists'][0]['name'])
        track_list.append(track_dict['name'])
        album_list.append(track_dict['album']['name'])
        popularity_list.append(track_dict['popularity'])
        
    # check if there are more pages
    if tracks['next']:
        tracks = sp.next(tracks)
    else:
        break

print (len(list(playlist_dict_by_str.keys())))
print (len(list(playlist_dict_by_uri.keys())))


Enter the URL you were redirected to: https://druce.ai/?code=AQByRJnyUJYYtjbuFQnbpsD-Nty9g8zw93dka8JwBOdKjKcpUhZ1CDh6lRJQXbH7wmni7F-xSq0BVUIh2EjJuxvCAltNBf2GNvwkNHmloU5BsTWvi_1oFDjntArWRChlAi9LWZxJGT4lCvwlicNpnWveNJrJy005IIyQvrXQrZ3FUmlxTrEujF64gbQ
Nine Inch Nails | A Warm Place
Joni Mitchell | Both Sides Now
1027
1029


In [71]:
playlist_df = pd.DataFrame({'artist': artist_list,
                           'track': track_list,
                           'album': album_list,
                           'popularity': popularity_list,
                           })



In [72]:
with pd.option_context("display.max_rows", 9999):
    display(playlist_df.sort_values('popularity'))
    

,artist,track,album,popularity
1028,Joni Mitchell,Both Sides Now,Clouds,0
883,Giacomo Puccini,"Tosca, S. 69, Act II: Vissi d'arte",TikTok Klassische Musik,0
790,Johann Sebastian Bach,"Jesu Joy of Man's Desiring, BWV 147 (Arr. Hess)",Klassik zum Studieren,0
787,Jaakko Aukusti,What If All Else Fails?,What If All Else Fails?,0
739,Frédéric Chopin,"Ballade No. 3 in A-Flat Major, Op. 47","Mozart, Chopin: Best Classical Music",0
731,Astrud Gilberto,Photograph,BRAZIL ALL STARS : BOSSA NOVA HITS,0
702,Pyotr Ilyich Tchaikovsky,"Romeo and Juliet, TH 42 ""Fantasy Overture""",Classical Love songs,0
686,Sergei Rachmaninoff,"14 Songs, Op. 34: No. 14, Vocalise",Piano Musik Zum Studieren,0
549,Edvard Grieg,"Peer Gynt, Op. 23, Act IV: Morning Mood",TikTok Klassische Musik,0
933,Yusuf / Cat Stevens,Wild World,The Best Of Cat Stevens 20th Century Masters T...,0


In [73]:
gold_dict_by_uri = {}
gold_dict_by_str = {}
addlist = []
c = 0
for i, artist, track, uri in gold_df[['artist', 'track', 'uri']].itertuples():
    # print(artist, track, uri)
    track_str = artist + ' | ' + track[:15]
    if track_str not in playlist_dict_by_str:
        addlist.append([artist, track, uri])
        print(artist, track, uri)
    gold_dict_by_uri[uri]=track_str
    gold_dict_by_str['track_str']= uri
#     if track_str not in playlist_dict_by_str:
#         c += 1
#         print (c, track_str)
        
print(len(gold_dict_by_str.items()))
print(len(gold_dict_by_uri.items()))

ABBA One Of Us spotify:track:6zgtBUEkAfilJ2YEOvNexR
Gregorio Allegri Miserere mei, Deus spotify:track:6es7DmrhnDoKj5rsFvh3XU
Amy Winehouse Love Is A Losing Game spotify:track:3uliGwmB52ZA7brgpZMzyH
Barbara Ma plus belle histoire d'amour spotify:track:0qBVET4VkHsQAoboWlQ2pJ
Ludwig van Beethoven Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio spotify:track:2ygeBLTP9uu3OW3VTulD8N
Benny Goodman Sing, Sing, Sing spotify:track:5L8ta4ECl5zeA6bGqY7G38
Bill Withers Lean on Me spotify:track:3M8FzayQWtkvOhqMn2V4T2
Billy Joel Piano Man spotify:track:70C4NyhjD5OZUMzvWZ3njJ
Bob Dylan Ballad of a Thin Man spotify:track:0f5N14nB8xi0p3o4BlVvbx
Bob Dylan Blowin' in the Wind spotify:track:18GiV1BaXzPVYpp9rmOg0E
Bob Dylan Desolation Row spotify:track:4n1ZGm3TxYmoYe1YR8cMus
Bob Dylan Duquesne Whistle spotify:track:5kKW4bszhKSCYVPDO0sMbX
Bob Dylan Forever Young - Slow Version spotify:track:4yWl0tnEanf3zmZzl9kbQn
Bob Dylan Gotta Serve Somebody spotify:track:760420tYNmNjFgi8bWvbop
Bob Dylan Highway 61 

In [74]:
addlist

[['ABBA', 'One Of Us', 'spotify:track:6zgtBUEkAfilJ2YEOvNexR'],
 ['Gregorio Allegri',
  'Miserere mei, Deus',
  'spotify:track:6es7DmrhnDoKj5rsFvh3XU'],
 ['Amy Winehouse',
  'Love Is A Losing Game',
  'spotify:track:3uliGwmB52ZA7brgpZMzyH'],
 ['Barbara',
  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Ludwig van Beethoven',
  'Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio',
  'spotify:track:2ygeBLTP9uu3OW3VTulD8N'],
 ['Benny Goodman', 'Sing, Sing, Sing', 'spotify:track:5L8ta4ECl5zeA6bGqY7G38'],
 ['Bill Withers', 'Lean on Me', 'spotify:track:3M8FzayQWtkvOhqMn2V4T2'],
 ['Billy Joel', 'Piano Man', 'spotify:track:70C4NyhjD5OZUMzvWZ3njJ'],
 ['Bob Dylan', 'Ballad of a Thin Man', 'spotify:track:0f5N14nB8xi0p3o4BlVvbx'],
 ['Bob Dylan', "Blowin' in the Wind", 'spotify:track:18GiV1BaXzPVYpp9rmOg0E'],
 ['Bob Dylan', 'Desolation Row', 'spotify:track:4n1ZGm3TxYmoYe1YR8cMus'],
 ['Bob Dylan', 'Duquesne Whistle', 'spotify:track:5kKW4bszhKSCYVPDO0sMbX'],
 ['Bo

In [76]:
addlist = [['ABBA', 'One Of Us', 'spotify:track:6zgtBUEkAfilJ2YEOvNexR'],
 ['Gregorio Allegri',
  'Miserere mei, Deus',
  'spotify:track:6es7DmrhnDoKj5rsFvh3XU'],
 ['Amy Winehouse',
  'Love Is A Losing Game',
  'spotify:track:3uliGwmB52ZA7brgpZMzyH'],
 ['Barbara',
  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Ludwig van Beethoven',
  'Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio',
  'spotify:track:2ygeBLTP9uu3OW3VTulD8N'],
 ['Benny Goodman', 'Sing, Sing, Sing', 'spotify:track:5L8ta4ECl5zeA6bGqY7G38'],
 ['Bill Withers', 'Lean on Me', 'spotify:track:3M8FzayQWtkvOhqMn2V4T2'],
 ['Billy Joel', 'Piano Man', 'spotify:track:70C4NyhjD5OZUMzvWZ3njJ'],
 ['Bob Dylan', 'Ballad of a Thin Man', 'spotify:track:0f5N14nB8xi0p3o4BlVvbx'],
 ['Bob Dylan', "Blowin' in the Wind", 'spotify:track:18GiV1BaXzPVYpp9rmOg0E'],
 ['Bob Dylan', 'Desolation Row', 'spotify:track:4n1ZGm3TxYmoYe1YR8cMus'],
 ['Bob Dylan', 'Duquesne Whistle', 'spotify:track:5kKW4bszhKSCYVPDO0sMbX'],
 ['Bob Dylan',
  'Forever Young - Slow Version',
  'spotify:track:4yWl0tnEanf3zmZzl9kbQn'],
 ['Bob Dylan', 'Gotta Serve Somebody', 'spotify:track:760420tYNmNjFgi8bWvbop'],
 ['Bob Dylan', 'Highway 61 Revisited', 'spotify:track:6os5B6xjuke9YfBKH3tu1e'],
 ['Bob Dylan',
  'I Shall Be Released - Studio Outtake - 1971',
  'spotify:track:5vyw005QQ42hrzrLxb3xEX'],
 ['Bob Dylan', 'I Want You', 'spotify:track:7tJQ4Ekp2vN3NlI3vJJW3v'],
 ['Bob Dylan', "It Ain't Me Babe", 'spotify:track:5nbNWAfT1S6V1vqj3snHxS'],
 ['Bob Dylan', 'Jokerman', 'spotify:track:6cuHkcRUqtQhtJ4sWCkd1q'],
 ['Bob Dylan',
  "Knockin' On Heaven's Door",
  'spotify:track:6HSXNV0b4M4cLJ7ljgVVeh'],
 ['Bob Dylan', 'Lay, Lady, Lay', 'spotify:track:4uYwlMp841PLJmj1gJJwIq'],
 ['Bob Dylan', 'Like a Rolling Stone', 'spotify:track:3AhXZa8sUQht0UEdBJgpGc'],
 ['Bob Dylan', 'Love Sick', 'spotify:track:3O1hpSOaJDW4SelgUG2XT3'],
 ['Bob Dylan', "Maggie's Farm", 'spotify:track:5rGD8FFgHw74cp3RPhucyg'],
 ['Bob Dylan',
  'Make You Feel My Love',
  'spotify:track:6rfGPGghQL7SJmZPXprXIc'],
 ['Bob Dylan',
  'Mississippi - Version 2',
  'spotify:track:6JWHNd8QMxTvojYkmZtKGI'],
 ['Bob Dylan', 'Mr. Tambourine Man', 'spotify:track:3RkQ3UwOyPqpIiIvGVewuU'],
 ['Bob Dylan', 'Murder Most Foul', 'spotify:track:1LfTvT9JPYuuZanwxLtZCr'],
 ['Bob Dylan', 'Not Dark Yet', 'spotify:track:1qbn6QrHG8XfnqVFKgNzKP'],
 ['Bob Dylan',
  'Rainy Day Women #12 & 35',
  'spotify:track:7BkAlVpGwXXl3sYNn5OoJ7'],
 ['Bob Dylan',
  'Sad-Eyed Lady of the Lowlands',
  'spotify:track:4jdtLLyEL7wY0TlCdMKhxq'],
 ['Bob Dylan', 'She Belongs to Me', 'spotify:track:2itBkHBUxGl4VfDj4HNyoD'],
 ['Bob Dylan',
  'Stuck Inside of Mobile with the Memphis Blues Again',
  'spotify:track:1NYTj6JEw3IOh4ggiBh82h'],
 ['Bob Dylan',
  'Subterranean Homesick Blues',
  'spotify:track:6k9DUKMJpWvu6eFG3O64Lg'],
 ['Bob Dylan', 'Tangled up in Blue', 'spotify:track:6Vcwr9tb3ZLO63F8DL8cqu'],
 ['Bob Dylan', 'Tempest', 'spotify:track:19scNzd4ogVsHrNWsms8Rg'],
 ['Bob Dylan',
  "The Times They Are A-Changin'",
  'spotify:track:52vA3CYKZqZVdQnzRrdZt6'],
 ['Bob Dylan',
  'Things Have Changed - Single Version',
  'spotify:track:5KOi77ameCimkAdw0DMNoy'],
 ['Bob Dylan',
  'Thunder on the Mountain',
  'spotify:track:4wo2eRp6aHcAlmhmfwiTAH'],
 ['Bob Dylan', 'Visions of Johanna', 'spotify:track:2rslQV48gNv3r9pPrQFPW1'],
 ['Brian Wilson', 'God Only Knows', 'spotify:track:2SznAUigFh6rMdGpcS5d7e'],
 ['Bright Eyes',
  'First Day of My Life',
  'spotify:track:0eBryM7ePQH3Klt3jz8xZd'],
 ['Crowded House',
  'Don’t Dream It’s Over - Home Demo',
  'spotify:track:0fiSpF9mvRFQWy0ca64d1g'],
 ['Léo Delibes', 'Flower Duet', 'spotify:track:5K8jqeLAxZIqHR6e5w5so1'],
 ['Dire Straits', 'Brothers In Arms', 'spotify:track:6XYBbVpu455ZdGWZNRLGbG'],
 ['Don McLean',
  'Vincent (Starry, Starry Night)',
  'spotify:track:2YDyH60Vro33KkDtNZCXIk'],
 ['Ed Sheeran', 'Photograph', 'spotify:track:41xNsY82OWtWbIfnRMK2ky'],
 ['Elvis Presley',
  'Can’t Help Falling in Love - Acoustic Cover',
  'spotify:track:0ghQkNDYLSl4GsqfkjTjWx'],
 ['Enya', 'Amarantine', 'spotify:track:0VmzazQQ0Mo1vJldr5NxTW'],
 ['Evan Rachel Wood', 'If I Fell', 'spotify:track:0gd3hRBQAEAw096YOcUrmR'],
 ['Fleetwood Mac', 'Rhiannon', 'spotify:track:05oETzWbd4SI33qK2gbJfR'],
 ['George Harrison',
  'All Things Must Pass - 2014 Remaster',
  'spotify:track:16OwZQuzMqnwn3FZsCBZly'],
 ['George Harrison',
  'Apple Scruffs - 2014 Remaster',
  'spotify:track:2K7WhpfZX3TCCMiwebp0W7'],
 ['George Harrison',
  'Art of Dying - 2014 Remaster',
  'spotify:track:6Jod7qrtYBhU3HcUmKk4hX'],
 ['George Harrison',
  'Awaiting on You All - 2014 Remaster',
  'spotify:track:0b65WkrBrg2qOkzQeDtQ9d'],
 ['George Harrison',
  'Ballad of Sir Frankie Crisp (Let It Roll) - 2014 Remaster',
  'spotify:track:0FWeRrB8T5R6maHbWQw4Kk'],
 ['George Harrison',
  'Behind That Locked Door',
  'spotify:track:2VVbLn8nMcWJzjcL1tZsUr'],
 ['George Harrison',
  'Beware of Darkness - 2014 Remaster',
  'spotify:track:606MCyZFMBlc52Ojnn1nvU'],
 ['George Harrison',
  'Give Me Love (Give Me Peace on Earth)',
  'spotify:track:71fXxvXqo1zxWDtBmjoEVk'],
 ['George Harrison',
  'Hear Me Lord - 2014 Remaster',
  'spotify:track:3kopbNyRj10XO1actGZexP'],
 ['George Harrison',
  'I Dig Love - 2014 Remaster',
  'spotify:track:42yK1Wy62c7malKSRwy0Qk'],
 ['George Harrison',
  'I Remember Jeep - 2014 Remaster',
  'spotify:track:058AE5M3ifbCh8VWOV7903'],
 ['George Harrison',
  "It's Johnny's Birthday - 2014 Remaster",
  'spotify:track:6Cv05rcW8HWwCC6wyEp1fC'],
 ['George Harrison',
  'Let It Down - 2014 Remaster',
  'spotify:track:5FFruMKbVg8AhwHnX4xBov'],
 ['George Harrison',
  'My Sweet Lord - 2014 Remaster',
  'spotify:track:6vE90mi4yKsQGY3YD2OOv1'],
 ['George Harrison',
  'Out of the Blue - 2014 Remaster',
  'spotify:track:1KHMyFaGvwVQ7ax4yjq4BZ'],
 ['George Harrison',
  'Plug Me In - 2014 Remaster',
  'spotify:track:0tyk2xHVjBd3nk16cGktTG'],
 ['George Harrison',
  'Run of the Mill - 2014 Remaster',
  'spotify:track:4uSlUBg3NVOA77E7wwKFTO'],
 ['George Harrison',
  'Thanks for the Pepperoni - 2014 Remaster',
  'spotify:track:3smkwfPqFsTmwfnBztMXaM'],
 ['George Harrison',
  'The Inner Light (Alternative Take) - Instrumental',
  'spotify:track:7gWPnvhaBFMlQsTBWEGcSC'],
 ['George Harrison',
  'Wah-Wah - 2014 Remaster',
  'spotify:track:5j3aqkMO2fl0s5eaSuVnQ8'],
 ['George Harrison',
  'What Is Life - 2014 Remaster',
  'spotify:track:44fw7RulJyj7dGIi9qR86N'],
 ['George Harrison',
  'While My Guitar Gently Weeps - Live At Madison Square Garden; 2009 Remaster',
  'spotify:track:4Egi6XuC0rbLlXfqmQeuFa'],
 ['Glenn Miller', 'In the Mood', 'spotify:track:1xsY8IFXUrxeet1Fcmk4oC'],
 ['Hans Zimmer', 'Cornfield Chase', 'spotify:track:6pWgRkpqVfxnj3WuIcJ7WP'],
 ['Hans Zimmer',
  'Day One (Interstellar Theme)',
  'spotify:track:4WmB04GBqS4xPMYN9dHgBw'],
 ["Israel Kamakawiwo'ole",
  'Maui Medley',
  'spotify:track:6TSJ3L9pBQsYIlCD5pk7ju'],
 ['James Taylor',
  'You’ve Got a Friend',
  'spotify:track:3nK4hWsTEr7fVXziI5bTmh'],
 ['Jay Ungar', 'Ashoken Farewell', 'spotify:track:2s6pqLeVialgt5l5TTSeas'],
 ['Jeff Buckley',
  'If You Knew - Live at Sin-é, New York, NY - July/August 1993',
  'spotify:track:1nd2JEHXbUuQFDiQzCBpsv'],
 ['Jimi Hendrix', 'One Rainy Wish', 'spotify:track:5Zyv0v4rPcrXjkaeImuodv'],
 ['Jimi Hendrix',
  'Spanish Castle Magic',
  'spotify:track:2KFE98Iw0X23sf4vJYcbLH'],
 ['Jimi Hendrix',
  'Wait Until Tomorrow',
  'spotify:track:2YtVzmZzew1ILUdNueyWd7'],
 ['John Lennon',
  'Imagine - Remastered 2010',
  'spotify:track:7pKfPomDEeI4TPT6EOYjn9'],
 ['John Mayer', 'Queen of California', 'spotify:track:0CETmgFGt8Ne8vLnaLcduU'],
 ['Johnny Cash',
  'I Walk The Line - Single Version',
  'spotify:track:1TKPfF2fvn6gVLVfp3iG4j'],
 ['Joni Mitchell',
  'Mitchell: Urge for Going (Instrumental Arrangement of the B-Side Track of the Joni Mitchell Single "You Turn Me on I\'m a Radio")',
  'spotify:track:1I1u9aTdxxQ7SDLgBB3V7b'],
 ['Kanye West', 'Come to Life', 'spotify:track:5xvXeuxISyXJDRbZZf4uzd'],
 ['Leonard Cohen', 'Chelsea Hotel #2', 'spotify:track:4krhCfJg0znykZoyjeMXRe'],
 ['Leonard Cohen', 'Dear Heather', 'spotify:track:3MTKMphPprAcBFG1uIhzPZ'],
 ['Leonard Cohen',
  "Death of a Ladies' Man",
  'spotify:track:5wrylUGwZugelovhryPYg2'],
 ['Leonard Cohen', 'The Future', 'spotify:track:5l8lYrnPEM1ln3J4XaTcy5'],
 ['Leonard Cohen',
  'You Want It Darker',
  'spotify:track:5zb7npjQqoJ7Kcpq4yD9qn'],
 ['Lingers.On', 'In Lingerie', 'spotify:track:6FH3kGlJbFVJDCG9RcERf7'],
 ['Louis Armstrong',
  'La vie en rose - Single Version',
  'spotify:track:3yYfoYGVpriV4fG9L1ogsD'],
 ['The Lovecats', 'The Lovecats', 'spotify:track:7iJUiiTfnuY5cTIeEBnqHr'],
 ['Ludovico Einaudi', 'Primavera', 'spotify:track:4BMHp3DkI8VLsuB9Kr0pzu'],
 ['Mazzy Star', 'Flowers In December', 'spotify:track:0G6Ws8Gbdt0S7pZeuYmkmm'],
 ['Metallica',
  'Fade To Black (Remastered)',
  'spotify:track:0dqGfCMAGyDgpUAgLNOjWd'],
 ['Wolfgang Amadeus Mozart',
  'Requiem in D Minor, K. 626: III. Sequenz No. 6, Lacrimosa dies illa',
  'spotify:track:4bvzJZXpkI3bkjxMCWOSu1'],
 ['My Chemical Romance',
  'The Light Behind Your Eyes',
  'spotify:track:3HyDpKAuR3e4l6QB7hSB2l'],
 ['Paul McCartney',
  'Here Today - Remixed 2015',
  'spotify:track:0QtnwXDziZN1K55fXuLN6q'],
 ['Paul McCartney',
  'I’ll Follow The Sun - Live At Amoeba 2007',
  'spotify:track:3xT59EeQdq0TPGtOlXXI8t'],
 ['Puscifer', 'The Humbling River', 'spotify:track:69GE6yPZZldvqtgBHrKXxg'],
 ['Ray LaMontagne',
  'Such A Simple Thing',
  'spotify:track:4PuUa8e5s7P3Zv1IdCGIsa'],
 ['Ray Manzarek',
  'Riders on the Storm',
  'spotify:track:3FvYcTXO2QtDY7kZQHku2d'],
 ['Red Hot Chili Peppers', 'Dosed', 'spotify:track:1iFIZUVDBCCkWe705FLXto'],
 ['Sky Cries Mary',
  "Don't Forget The Sky",
  'spotify:track:4sVpjCJRClVetRrdxVBolP'],
 ['Stevie Nicks', 'Landslide', 'spotify:track:5fprEY6WEN1wvFXkgfb22C'],
 ['Stevie Wonder', 'Isn’t She Lovely', 'spotify:track:6wGlAaMfyhKdEPr2zycAnN'],
 ['Taylor Swift',
  'Fearless (Taylor’s Version)',
  'spotify:track:77sMIMlNaSURUAXq5coCxE'],
 ['Taylor Swift',
  'the lakes - bonus track',
  'spotify:track:0eFQWVz0qIxDOvhLpZ40P7'],
 ['The Band',
  'When I Paint My Masterpiece - Remastered',
  'spotify:track:76WChUuOPeIK027IeUgr0l'],
 ['The Beach Boys',
  "I Just Wasn't Made For These Times - Mono",
  'spotify:track:4CuO8TINNqM3D7aUdNQ3zG'],
 ['The Beach Boys',
  "Let's Go Away For A While - Mono",
  'spotify:track:3GsgJI1aBrvUtqX8f3MhKT'],
 ['The Beatles',
  "Don't Let Me Down - Naked Version / Remastered 2013",
  'spotify:track:5BhMoGrz5KzG2fA5uzHjZ1'],
 ['The Beatles',
  'Love Me Do - Remastered 2009',
  'spotify:track:3VbGCXWRiouAq8VyMYN2MI'],
 ['The Chemical Brothers',
  'The Boxer',
  'spotify:track:1EUeDFq2zNP784GPaRs9aH'],
 ['The Cure',
  'A Night like This - 2006 Remaster',
  'spotify:track:7cKCz7gG84i1XLvDeM3ByT'],
 ['The Cure',
  'Disintegration - 2010 Remaster',
  'spotify:track:0zY8t5dC1KQXcPUKByWMJM'],
 ['The Cure',
  'From the Edge of the Deep Green Sea',
  'spotify:track:2vwBL9RVyr0vA4Og5VH0i3'],
 ['The Cure',
  'In Between Days - 2006 Remaster',
  'spotify:track:07CyrZF9eVd02zzIse7tZA'],
 ['The Cure', 'A Letter to Elise', 'spotify:track:4DdXOLc1VMAY34ourCn1Xa'],
 ['The Cure',
  'Lullaby - 2010 Remaster',
  'spotify:track:4d4oXk7O2lEhZ83ivV93li'],
 ['The Cure', 'Underneath The Stars', 'spotify:track:0PKVjYlKw7z3IvKAoxrYTR'],
 ['The Eagles', 'The Desperadoes', 'spotify:track:10ppF835WJMYI5v65gFLZ3'],
 ['The Helio Sequence',
  'Keep Your Eyes Ahead',
  'spotify:track:3yatRBsGMJ7wMoUIgDBzzo'],
 ['The Moldy Peaches',
  'Anyone Else But You',
  'spotify:track:2pKi1lRvXNASy7ybeQIDTy'],
 ['The Strokes', 'Someday', 'spotify:track:7hm4HTk9encxT0LYC0J6oI'],
 ['Traditional',
  'Scarborough Fair (Arr. Parkin)',
  'spotify:track:4wlNPczIullwvmwb4x0ltz'],
 ['Van Morrison',
  'Madame George - 1999 Remaster',
  'spotify:track:1N4MKISvC1ddfRCRQDXDd2'],
 ['Various Artists',
  'The Girl From Ipanema',
  'spotify:track:0JgH7g0kwsIs1THEVqhlUS'],
 ['Víg Mihály',
  'Öreg - From "Werckmeister Harmóniák"',
  'spotify:track:63wMgkXQuomlkW4an4O9b4'],
 ['Willie Nelson', 'Crazy', 'spotify:track:0xqtcLB45iKNfHroi5y1em']]


In [77]:
len(addlist)

134

In [ ]:
addlist2 = [a[2] for a in addlist]

print (len(addlist2), 'items')

while(addlist2):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist2[-100:])
    addlist2 = addlist2[:-100]
    print("added items, remaining ", len(addlist2))
